In [13]:
import pynwb
from allensdk.brain_observatory.ecephys.dynamic_gating_ecephys_session import DynamicGatingEcephysSession
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
%matplotlib inline

import sys

# Add the path you want to include
sys.path.append('aperiodic_shield/code')

from shield_utils import find_animals, get_lfp_dict, downsample, align_lfp

import json

data_path = '/oscar/data/sjones/kduecker/shield_data/'

meta_path = 'aperiodic_shield/externals/SHIELD_Dynamic_Gating_Analysis'


In [2]:
# find mice of interest

mice = find_animals(meta_path)


In [3]:
with open('aperiodic_shield/session_info.json', 'r') as file:
    session_info = json.load(file)

In [4]:
subj = session_info['sessions_all_areas'][0][0:6]

In [5]:
# get good subjects and good sessions from Christian's .json file
ses_files = os.listdir(os.path.join(data_path,f'sub-{subj}'))

# get lfp files and spike files
lfp_files = list(filter(lambda s: 'None' in s, ses_files))
session = session_info['sessions_all_areas'][0][7:-1]
spike_files = list(filter(lambda s: session in s, ses_files))


In [15]:
# roi=['LGd', 'VISp', 'VISl']

# nwb_file_asset = pynwb.NWBHDF5IO(f'{data_path}/sub-{subj}/{spike_files[0]}', mode='r', load_namespaces=True)
# nwb_file = nwb_file_asset.read()
# dynamic_gating_session = DynamicGatingEcephysSession.from_nwb(nwb_file)

# # probe map
# probe_index = dynamic_gating_session.probes.index
# probe_map = {}
# for p in probe_index:
#     probe_name = dynamic_gating_session.probes.name[p]
#     filename = list(filter(lambda s: '-'+str(p)+'_' in s, lfp_files))
#     probe_map[probe_name] = os.path.join(os.path.join(data_path,f'sub-{subj}'),filename[0])

# # add the LFP data to the session object
# dynamic_gating_session = DynamicGatingEcephysSession.from_nwb(nwb_file, probe_data_path_map=probe_map)

# # get the channels
# sess_units = dynamic_gating_session.get_units()

# # find the different layers in the brain area (e.g. VISpl2/3. VISl4, VISpl5)

# areas = np.unique(sess_units.structure_layer.values)
# area_layers = [name for name in areas if any(r in name for r in roi)]

# # extract layer for each unit
# layer_areas_units = dict()

# for al in area_layers:
#     layer_areas_units[al] = sess_units[sess_units.structure_layer.str.contains(al)].index

# # get stimulus presentations
# stim_presentations = dynamic_gating_session.stimulus_presentations
# flashes = stim_presentations[stim_presentations['stimulus_name'].str.contains('flash')]
# presentation_times = flashes.start_time.values
# flash_end_times = presentation_times + flashes.duration
# presentation_ids = flashes.index.values

# dt = 1/dynamic_gating_session.probes.sampling_rate[0]

# # organize LFP for each region/layer in dictionary
# layer_data = dict()

# # loop over probes
# # for pi in probe_index[0]:
# pi = 0
# lfp = dynamic_gating_session.get_lfp(pi)

#     # align LFP to stimulus presentations
# toi = [0, 1]
# aligned_lfp = align_lfp(lfp, np.arange(toi[0], toi[1], dt), presentation_times, presentation_ids)

#     # # extract lfp in the different brain areas
#     # lfp_channel_layer = dict()
#     # for layer, channels in zip(layer_areas_units.keys(), layer_areas_units.values()):
#     #     lfp_channel_layer[layer] = [chan for chan in aligned_lfp.channel.values if chan in channels]


#     # for layer, channels in lfp_channel_layer.items():
#     #     if channels:  # Only process layers with channels
#     #         layer_data[layer] = aligned_lfp.sel(channel=channels)

/oscar/home/kduecker/aperiodic_shield/code/shield_utils.py:52: FutureWarning: the `pandas.MultiIndex` object(s) passed as 'time' coordinate(s) or data variable(s) will no longer be implicitly promoted and wrapped into multiple indexed coordinates in the future (i.e., one coordinate for each multi-index level + one dimension coordinate). If you want to keep this behavior, you need to first wrap it explicitly using `mindex_coords = xarray.Coordinates.from_pandas_multiindex(mindex_obj, 'dim')` and pass it as coordinates, e.g., `xarray.Dataset(coords=mindex_coords)`, `dataset.assign_coords(mindex_coords)` or `dataarray.assign_coords(mindex_coords)`.
  ds = ds.assign(time=inds).unstack('time')


In [16]:
# aligned_lfp

<xarray.DataArray 'aligned_lfp' (channel: 96, presentation_id: 150,
                                 time_from_presentation_onset: 30001)> Size: 2GB
array([[[-2.46674990e-05, -2.46674990e-05, -2.46674990e-05, ...,
          9.67199958e-05,  9.67199958e-05,  9.67199958e-05],
        [ 4.41675002e-05,  4.41675002e-05,  4.41675002e-05, ...,
          4.19249991e-05,  4.19249991e-05,  4.19249991e-05],
        [ 7.40999985e-06,  7.40999985e-06,  7.40999985e-06, ...,
          1.84274995e-05,  1.84274995e-05,  1.84274995e-05],
        ...,
        [ 2.54474999e-05,  2.54474999e-05,  2.54474999e-05, ...,
          6.40575017e-05,  6.40575017e-05,  6.40575017e-05],
        [-1.30357497e-04, -1.30357497e-04, -1.30357497e-04, ...,
          2.43750001e-05,  2.43750001e-05,  2.43750001e-05],
        [-1.53464993e-04, -1.53464993e-04, -1.53464993e-04, ...,
         -6.26925030e-05, -6.26925030e-05, -6.26925030e-05]],

       [[-4.02674996e-05, -4.02674996e-05, -4.02674996e-05, ...,
          9.98399992e-05,  9.98399992e-05,  9.98399992e-05],
        [ 3.35400000e-05,  3.35400000e-05,  3.35400000e-05, ...,
          8.47275005e-05,  8.47275005e-05,  8.47275005e-05],
        [ 3.65624983e-05,  3.65624983e-05,  3.65624983e-05, ...,
          1.49175003e-05,  1.49175003e-05,  1.49175003e-05],
...
          2.63249990e-06,  2.63249990e-06,  2.63249990e-06],
        [-7.40999985e-06, -7.40999985e-06, -7.40999985e-06, ...,
         -8.96999973e-06, -8.96999973e-06, -8.96999973e-06],
        [-3.75374984e-05, -3.75374984e-05, -3.75374984e-05, ...,
         -2.63249990e-06, -2.63249990e-06, -2.63249990e-06]],

       [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]]],
      dtype=float32)
Coordinates:
  * presentation_id               (presentation_id) int64 1kB 8847 8848 ... 8996
  * time_from_presentation_onset  (time_from_presentation_onset) float64 240kB ...
  * channel                       (channel) int64 768B 2 6 10 14 ... 374 378 382

In [17]:
layer_lfp = get_lfp_dict(subj, data_path, lfp_files, spike_files[0], toi=[0, 1])

In [ ]:
layer_lfp

{'VISp2/3': <xarray.DataArray 'aligned_lfp' (channel: 4, presentation_id: 150,
                                  time_from_presentation_onset: 30001)> Size: 72MB
 array([[[ 8.29724959e-05,  8.29724959e-05,  8.29724959e-05, ...,
          -1.91100000e-04, -1.91100000e-04, -1.91100000e-04],
         [ 1.05884996e-04,  1.05884996e-04,  7.98524998e-05, ...,
          -2.17815003e-04, -2.17815003e-04, -2.17815003e-04],
         [ 4.54739988e-04,  4.54739988e-04,  4.54739988e-04, ...,
           1.65847494e-04,  1.65847494e-04,  2.14109998e-04],
         ...,
         [ 1.83007505e-04,  1.83007505e-04,  1.83007505e-04, ...,
           2.69879994e-04,  2.69879994e-04,  2.69879994e-04],
         [ 1.08517503e-04,  1.08517503e-04,  1.08615000e-04, ...,
           2.45504983e-04,  2.45504983e-04,  2.45504983e-04],
         [ 1.66725004e-05,  1.66725004e-05,  1.66725004e-05, ...,
          -5.51265024e-04, -5.51265024e-04, -5.51265024e-04]],
 
        [[ 9.82799975e-05,  9.82799975e-05,  9.827999